In [2]:
import numpy as np

class ANN:
    """
    Feedforward Artificial Neural Network with configurable architecture.
    Implements multiple activation functions for regression tasks.
    """

    def __init__(self, layer_sizes, activation_functions):
        """
        Initialize ANN with specified architecture.

        Args:
            layer_sizes: List of integers specifying neurons in each layer
                        e.g., [8, 10, 5, 1] for 8 inputs, 2 hidden layers, 1 output
            activation_functions: List of activation function names for each layer
                                 e.g., ['relu', 'relu', 'linear']
        """
        self.layer_sizes = layer_sizes
        self.num_layers = len(layer_sizes)
        self.activation_functions = activation_functions

        # Initialize weights and biases randomly
        self.weights = []
        self.biases = []

        # Create weight matrices between consecutive layers
        for i in range(self.num_layers - 1):
            # Xavier initialization for better convergence
            limit = np.sqrt(6 / (layer_sizes[i] + layer_sizes[i+1]))
            w = np.random.uniform(-limit, limit, (layer_sizes[i], layer_sizes[i+1]))
            b = np.zeros((1, layer_sizes[i+1]))
            self.weights.append(w)
            self.biases.append(b)

    def set_parameters(self, params):
        """
        Set network parameters (weights and biases) from a flat vector.
        Used by PSO to update the network.

        Args:
            params: 1D numpy array containing all weights and biases
        """
        idx = 0
        self.weights = []
        self.biases = []

        # Reconstruct weight matrices and bias vectors from flat array
        for i in range(self.num_layers - 1):
            # Calculate size of weight matrix
            w_size = self.layer_sizes[i] * self.layer_sizes[i+1]
            # Extract and reshape weights
            w = params[idx:idx+w_size].reshape(self.layer_sizes[i], self.layer_sizes[i+1])
            idx += w_size

            # Extract biases
            b_size = self.layer_sizes[i+1]
            b = params[idx:idx+b_size].reshape(1, self.layer_sizes[i+1])
            idx += b_size

            self.weights.append(w)
            self.biases.append(b)

    def get_parameter_count(self):
        """
        Calculate total number of parameters (weights + biases) in the network.

        Returns:
            Integer count of total parameters
        """
        count = 0
        for i in range(self.num_layers - 1):
            # Weights: input_size * output_size
            count += self.layer_sizes[i] * self.layer_sizes[i+1]
            # Biases: output_size
            count += self.layer_sizes[i+1]
        return count

    def activate(self, x, function_name):
        """
        Apply activation function to input.

        Args:
            x: Input array
            function_name: Name of activation function ('logistic', 'relu', 'tanh', 'linear')

        Returns:
            Activated output
        """
        if function_name == 'logistic':
            # Logistic sigmoid: 1 / (1 + e^(-x))
            return 1 / (1 + np.exp(-np.clip(x, -500, 500)))  # Clip to prevent overflow
        elif function_name == 'relu':
            # ReLU: max(0, x)
            return np.maximum(0, x)
        elif function_name == 'tanh':
            # Hyperbolic tangent
            return np.tanh(x)
        elif function_name == 'linear':
            # Linear (no activation)
            return x
        else:
            raise ValueError(f"Unknown activation function: {function_name}")

    def forward(self, X):
        """
        Perform forward propagation through the network.

        Args:
            X: Input data, shape (n_samples, n_features)

        Returns:
            Network output, shape (n_samples, n_outputs)
        """
        activation = X

        # Propagate through each layer
        for i in range(len(self.weights)):
            # Linear transformation: z = activation * weights + bias
            z = np.dot(activation, self.weights[i]) + self.biases[i]
            # Apply activation function
            activation = self.activate(z, self.activation_functions[i])

        return activation

    def predict(self, X):
        """
        Make predictions on input data.

        Args:
            X: Input data

        Returns:
            Predictions
        """
        return self.forward(X)
        import numpy as np

class PSO:
    """
    Particle Swarm Optimization implementation following Algorithm 39
    from "Essentials of Metaheuristics" by Sean Luke.
    Uses informants topology rather than global best.
    """

    def __init__(self, fitness_function, dimensions, bounds,
                 swarm_size=30, max_iterations=100,
                 num_informants=3, w=0.729, c1=1.49445, c2=1.49445):
        """
        Initialize PSO with specified parameters.

        Args:
            fitness_function: Function to minimize (lower is better)
            dimensions: Number of dimensions in solution space
            bounds: Tuple of (min, max) for parameter values
            swarm_size: Number of particles in swarm
            max_iterations: Maximum number of iterations
            num_informants: Number of informants per particle
            w: Inertia weight
            c1: Cognitive coefficient (personal best influence)
            c2: Social coefficient (neighborhood best influence)
        """
        self.fitness_function = fitness_function
        self.dimensions = dimensions
        self.bounds = bounds
        self.swarm_size = swarm_size
        self.max_iterations = max_iterations
        self.num_informants = num_informants
        self.w = w  # Inertia weight
        self.c1 = c1  # Cognitive coefficient
        self.c2 = c2  # Social coefficient

        # For tracking best solution found
        self.global_best_position = None
        self.global_best_fitness = float('inf')
        self.fitness_history = []

    def initialize_swarm(self):
        """
        Initialize particle positions and velocities.
        Corresponds to lines 1-2 of Algorithm 39.

        Returns:
            positions: Random initial positions
            velocities: Random initial velocities
        """
        # Line 1: Initialize positions uniformly in search space
        positions = np.random.uniform(
            self.bounds[0],
            self.bounds[1],
            (self.swarm_size, self.dimensions)
        )

        # Line 2: Initialize velocities
        velocity_range = (self.bounds[1] - self.bounds[0]) * 0.1
        velocities = np.random.uniform(
            -velocity_range,
            velocity_range,
            (self.swarm_size, self.dimensions)
        )

        return positions, velocities

    def create_informants(self):
        """
        Create informant topology (ring topology).
        Each particle is influenced by k neighbors.
        Corresponds to line 3 of Algorithm 39.

        Returns:
            List of lists, where informants[i] contains indices of particle i's informants
        """
        # Line 3: Assign informants to particles
        informants = []
        for i in range(self.swarm_size):
            # Ring topology: include neighbors on both sides
            neighbors = []
            for j in range(1, self.num_informants + 1):
                left = (i - j) % self.swarm_size
                right = (i + j) % self.swarm_size
                neighbors.append(left)
                if len(neighbors) < self.num_informants:
                    neighbors.append(right)
            # Include self and limit to num_informants
            neighbors = list(set([i] + neighbors[:self.num_informants-1]))
            informants.append(neighbors)
        return informants

    def optimize(self, verbose=True):
        """
        Main PSO optimization loop.
        Follows Algorithm 39 from Essentials of Metaheuristics.

        Returns:
            best_position: Best solution found
            best_fitness: Fitness of best solution
        """
        # Lines 1-3: Initialize swarm
        positions, velocities = self.initialize_swarm()
        informants = self.create_informants()

        # Line 4: Initialize personal bests
        personal_best_positions = positions.copy()
        personal_best_fitness = np.array([self.fitness_function(p) for p in positions])

        # Initialize neighborhood bests
        neighborhood_best_positions = np.zeros_like(positions)
        neighborhood_best_fitness = np.full(self.swarm_size, float('inf'))

        # Track global best
        best_idx = np.argmin(personal_best_fitness)
        self.global_best_position = personal_best_positions[best_idx].copy()
        self.global_best_fitness = personal_best_fitness[best_idx]

        # Line 5: Main loop
        for iteration in range(self.max_iterations):

            # Update neighborhood bests for each particle
            for i in range(self.swarm_size):
                # Find best among informants
                informant_fitness = [personal_best_fitness[j] for j in informants[i]]
                best_informant_idx = informants[i][np.argmin(informant_fitness)]
                neighborhood_best_positions[i] = personal_best_positions[best_informant_idx]
                neighborhood_best_fitness[i] = personal_best_fitness[best_informant_idx]

            # Update each particle
            for i in range(self.swarm_size):

                # Line 6: Generate random values
                rp = np.random.uniform(0, 1, self.dimensions)
                rg = np.random.uniform(0, 1, self.dimensions)

                # Line 7: Update velocity
                # v_i = w * v_i + c1 * rp * (p_i - x_i) + c2 * rg * (n_i - x_i)
                cognitive = self.c1 * rp * (personal_best_positions[i] - positions[i])
                social = self.c2 * rg * (neighborhood_best_positions[i] - positions[i])
                velocities[i] = self.w * velocities[i] + cognitive + social

                # Line 8: Update position
                positions[i] = positions[i] + velocities[i]

                # Line 9: Enforce bounds (clamping strategy)
                positions[i] = np.clip(positions[i], self.bounds[0], self.bounds[1])

                # Line 10: Evaluate fitness
                fitness = self.fitness_function(positions[i])

                # Line 11: Update personal best
                if fitness < personal_best_fitness[i]:
                    personal_best_positions[i] = positions[i].copy()
                    personal_best_fitness[i] = fitness

                    # Update global best if necessary
                    if fitness < self.global_best_fitness:
                        self.global_best_position = positions[i].copy()
                        self.global_best_fitness = fitness

            # Store best fitness for this iteration
            self.fitness_history.append(self.global_best_fitness)

            if verbose and (iteration + 1) % 10 == 0:
                print(f"Iteration {iteration + 1}/{self.max_iterations}, "
                      f"Best Fitness: {self.global_best_fitness:.4f}")

        return self.global_best_position, self.global_best_fitness
        import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import json

# Import the ANN and PSO classes (assuming they're in separate files)
# from ann import ANN
# from pso import PSO

def load_and_preprocess_data(filepath='Concrete_Data.csv'):
    """
    Load and preprocess the concrete strength dataset.

    Args:
        filepath: Path to CSV file

    Returns:
        X_train, X_test, y_train, y_test, scaler_X, scaler_y
    """
    # Load data
    data = pd.read_csv(filepath)

    # Separate features and target
    X = data.iloc[:, :-1].values  # First 8 columns are features
    y = data.iloc[:, -1].values.reshape(-1, 1)  # Last column is target

    # Split into training (70%) and testing (30%)
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.3, random_state=42
    )

    # Normalize features and target for better ANN performance
    scaler_X = StandardScaler()
    scaler_y = StandardScaler()

    X_train = scaler_X.fit_transform(X_train)
    X_test = scaler_X.transform(X_test)
    y_train = scaler_y.fit_transform(y_train)
    y_test = scaler_y.transform(y_test)

    return X_train, X_test, y_train, y_test, scaler_X, scaler_y


def mean_absolute_error(y_true, y_pred):
    """
    Calculate Mean Absolute Error.

    Args:
        y_true: True values
        y_pred: Predicted values

    Returns:
        MAE value
    """
    return np.mean(np.abs(y_true - y_pred))


def create_fitness_function(ann, X_train, y_train):
    """
    Create fitness function for PSO that evaluates ANN performance.
    Couples PSO and ANN together.

    Args:
        ann: ANN instance
        X_train: Training features
        y_train: Training targets

    Returns:
        Fitness function that takes parameter vector and returns MAE
    """
    def fitness(params):
        """
        Fitness function evaluates how well the ANN performs with given parameters.
        Lower MAE = better fitness.

        Args:
            params: Flat vector of ANN weights and biases

        Returns:
            Mean Absolute Error on training set
        """
        # Set ANN parameters from PSO particle
        ann.set_parameters(params)

        # Get predictions
        predictions = ann.predict(X_train)

        # Calculate and return error (fitness to minimize)
        mae = mean_absolute_error(y_train, predictions)
        return mae

    return fitness


def evaluate_ann(ann, X_test, y_test):
    """
    Evaluate trained ANN on test set.

    Args:
        ann: Trained ANN
        X_test: Test features
        y_test: Test targets

    Returns:
        Test MAE
    """
    predictions = ann.predict(X_test)
    mae = mean_absolute_error(y_test, predictions)
    return mae


def run_single_experiment(layer_sizes, activations, swarm_size, max_iterations,
                         w, c1, c2, num_informants, X_train, y_train, X_test, y_test):
    """
    Run a single PSO-ANN experiment.

    Args:
        layer_sizes: ANN architecture
        activations: Activation functions for each layer
        swarm_size: PSO swarm size
        max_iterations: PSO iterations
        w, c1, c2: PSO coefficients
        num_informants: Number of informants
        X_train, y_train: Training data
        X_test, y_test: Test data

    Returns:
        Dictionary with results
    """
    # Create ANN
    ann = ANN(layer_sizes, activations)

    # Get parameter count and bounds
    param_count = ann.get_parameter_count()
    bounds = (-2, 2)  # Typical range for neural network weights

    # Create fitness function
    fitness_func = create_fitness_function(ann, X_train, y_train)

    # Initialize PSO
    pso = PSO(
        fitness_function=fitness_func,
        dimensions=param_count,
        bounds=bounds,
        swarm_size=swarm_size,
        max_iterations=max_iterations,
        num_informants=num_informants,
        w=w,
        c1=c1,
        c2=c2
    )

    # Run optimization
    best_params, train_mae = pso.optimize(verbose=False)

    # Set best parameters and evaluate on test set
    ann.set_parameters(best_params)
    test_mae = evaluate_ann(ann, X_test, y_test)

    return {
        'train_mae': train_mae,
        'test_mae': test_mae,
        'fitness_history': pso.fitness_history
    }


def experiment_architecture(X_train, y_train, X_test, y_test, num_runs=10):
    """
    Experiment 1: Test different ANN architectures.

    Returns:
        Dictionary with results
    """
    print("\n" + "="*60)
    print("EXPERIMENT 1: ANN Architecture Investigation")
    print("="*60)

    # Different architectures to test
    architectures = [
        ([8, 5, 1], ['relu', 'linear'], '8-5-1 (ReLU)'),
        ([8, 10, 1], ['relu', 'linear'], '8-10-1 (ReLU)'),
        ([8, 20, 1], ['relu', 'linear'], '8-20-1 (ReLU)'),
        ([8, 10, 5, 1], ['relu', 'relu', 'linear'], '8-10-5-1 (ReLU-ReLU)'),
        ([8, 10, 1], ['tanh', 'linear'], '8-10-1 (Tanh)'),
        ([8, 10, 1], ['logistic', 'linear'], '8-10-1 (Logistic)'),
    ]

    results = {}

    for layers, activations, name in architectures:
        print(f"\nTesting architecture: {name}")
        train_maes = []
        test_maes = []

        for run in range(num_runs):
            result = run_single_experiment(
                layer_sizes=layers,
                activations=activations,
                swarm_size=30,
                max_iterations=50,
                w=0.729,
                c1=1.49445,
                c2=1.49445,
                num_informants=3,
                X_train=X_train,
                y_train=y_train,
                X_test=X_test,
                y_test=y_test
            )
            train_maes.append(result['train_mae'])
            test_maes.append(result['test_mae'])
            print(f"  Run {run+1}/{num_runs}: Test MAE = {result['test_mae']:.4f}")

        results[name] = {
            'train_mae_mean': np.mean(train_maes),
            'train_mae_std': np.std(train_maes),
            'test_mae_mean': np.mean(test_maes),
            'test_mae_std': np.std(test_maes)
        }

        print(f"  Average Test MAE: {results[name]['test_mae_mean']:.4f} ± {results[name]['test_mae_std']:.4f}")

    return results


def experiment_swarm_allocation(X_train, y_train, X_test, y_test, num_runs=10):
    """
    Experiment 2: Test different swarm size vs iterations allocations.
    Fixed budget of 500 evaluations.

    Returns:
        Dictionary with results
    """
    print("\n" + "="*60)
    print("EXPERIMENT 2: Swarm Size vs Iterations (Budget=500)")
    print("="*60)

    # Different allocations with budget of 500
    allocations = [
        (10, 50, '10x50'),
        (25, 20, '25x20'),
        (50, 10, '50x10'),
        (100, 5, '100x5'),
    ]

    results = {}

    for swarm_size, iterations, name in allocations:
        print(f"\nTesting allocation: {name} (swarm={swarm_size}, iter={iterations})")
        train_maes = []
        test_maes = []

        for run in range(num_runs):
            result = run_single_experiment(
                layer_sizes=[8, 10, 1],
                activations=['relu', 'linear'],
                swarm_size=swarm_size,
                max_iterations=iterations,
                w=0.729,
                c1=1.49445,
                c2=1.49445,
                num_informants=3,
                X_train=X_train,
                y_train=y_train,
                X_test=X_test,
                y_test=y_test
            )
            train_maes.append(result['train_mae'])
            test_maes.append(result['test_mae'])
            print(f"  Run {run+1}/{num_runs}: Test MAE = {result['test_mae']:.4f}")

        results[name] = {
            'train_mae_mean': np.mean(train_maes),
            'train_mae_std': np.std(train_maes),
            'test_mae_mean': np.mean(test_maes),
            'test_mae_std': np.std(test_maes)
        }

        print(f"  Average Test MAE: {results[name]['test_mae_mean']:.4f} ± {results[name]['test_mae_std']:.4f}")

    return results


def experiment_coefficients(X_train, y_train, X_test, y_test, num_runs=10):
    """
    Experiment 3: Test different PSO acceleration coefficients.

    Returns:
        Dictionary with results
    """
    print("\n" + "="*60)
    print("EXPERIMENT 3: PSO Acceleration Coefficients")
    print("="*60)

    # Different coefficient combinations
    coefficient_sets = [
        (0.729, 1.49445, 1.49445, 'Standard (w=0.729, c1=c2=1.49)'),
        (0.5, 2.0, 2.0, 'High acceleration (w=0.5, c1=c2=2.0)'),
        (0.9, 2.0, 0.5, 'Cognitive-heavy (w=0.9, c1=2.0, c2=0.5)'),
        (0.9, 0.5, 2.0, 'Social-heavy (w=0.9, c1=0.5, c2=2.0)'),
        (0.4, 1.0, 1.0, 'Low inertia (w=0.4, c1=c2=1.0)'),
    ]

    results = {}

    for w, c1, c2, name in coefficient_sets:
        print(f"\nTesting: {name}")
        train_maes = []
        test_maes = []

        for run in range(num_runs):
            result = run_single_experiment(
                layer_sizes=[8, 10, 1],
                activations=['relu', 'linear'],
                swarm_size=30,
                max_iterations=50,
                w=w,
                c1=c1,
                c2=c2,
                num_informants=3,
                X_train=X_train,
                y_train=y_train,
                X_test=X_test,
                y_test=y_test
            )
            train_maes.append(result['train_mae'])
            test_maes.append(result['test_mae'])
            print(f"  Run {run+1}/{num_runs}: Test MAE = {result['test_mae']:.4f}")

        results[name] = {
            'train_mae_mean': np.mean(train_maes),
            'train_mae_std': np.std(train_maes),
            'test_mae_mean': np.mean(test_maes),
            'test_mae_std': np.std(test_maes)
        }

        print(f"  Average Test MAE: {results[name]['test_mae_mean']:.4f} ± {results[name]['test_mae_std']:.4f}")

    return results


def save_results(results, filename):
    """Save results to JSON file."""
    with open(filename, 'w') as f:
        json.dump(results, f, indent=2)
    print(f"\nResults saved to {filename}")


def main():
    """
    Main execution function.
    """
    print("Loading and preprocessing data...")
    X_train, X_test, y_train, y_test, scaler_X, scaler_y = load_and_preprocess_data()

    print(f"Training samples: {X_train.shape[0]}")
    print(f"Testing samples: {X_test.shape[0]}")
    print(f"Features: {X_train.shape[1]}")

    # Run experiments
    results = {}

    # Experiment 1: Architecture
    results['architecture'] = experiment_architecture(X_train, y_train, X_test, y_test, num_runs=10)

    # Experiment 2: Swarm allocation
    results['swarm_allocation'] = experiment_swarm_allocation(X_train, y_train, X_test, y_test, num_runs=10)

    # Experiment 3: Coefficients
    results['coefficients'] = experiment_coefficients(X_train, y_train, X_test, y_test, num_runs=10)

    # Save all results
    save_results(results, 'experiment_results.json')

    print("\n" + "="*60)
    print("ALL EXPERIMENTS COMPLETED")
    print("="*60)


if __name__ == "__main__":
    main()
    import matplotlib.pyplot as plt
import numpy as np
import json

def plot_convergence(fitness_history, title="PSO Convergence"):
    """
    Plot PSO convergence curve.

    Args:
        fitness_history: List of best fitness values per iteration
        title: Plot title
    """
    plt.figure(figsize=(10, 6))
    plt.plot(fitness_history, linewidth=2)
    plt.xlabel('Iteration', fontsize=12)
    plt.ylabel('Best Fitness (MAE)', fontsize=12)
    plt.title(title, fontsize=14, fontweight='bold')
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.savefig(f'{title.replace(" ", "_")}.png', dpi=300)
    plt.show()


def plot_architecture_comparison(results):
    """
    Plot comparison of different ANN architectures.

    Args:
        results: Dictionary with architecture experiment results
    """
    names = list(results.keys())
    test_means = [results[name]['test_mae_mean'] for name in names]
    test_stds = [results[name]['test_mae_std'] for name in names]

    plt.figure(figsize=(12, 6))
    x = np.arange(len(names))
    plt.bar(x, test_means, yerr=test_stds, capsize=5, alpha=0.7, color='steelblue')
    plt.xlabel('Architecture', fontsize=12)
    plt.ylabel('Test MAE', fontsize=12)
    plt.title('ANN Architecture Comparison', fontsize=14, fontweight='bold')
    plt.xticks(x, names, rotation=45, ha='right')
    plt.grid(True, alpha=0.3, axis='y')
    plt.tight_layout()
    plt.savefig('architecture_comparison.png', dpi=300)
    plt.show()


def plot_swarm_allocation(results):
    """
    Plot comparison of swarm size vs iterations allocations.

    Args:
        results: Dictionary with swarm allocation experiment results
    """
    names = list(results.keys())
    test_means = [results[name]['test_mae_mean'] for name in names]
    test_stds = [results[name]['test_mae_std'] for name in names]

    plt.figure(figsize=(10, 6))
    x = np.arange(len(names))
    plt.bar(x, test_means, yerr=test_stds, capsize=5, alpha=0.7, color='coral')
    plt.xlabel('Allocation (Swarm Size × Iterations)', fontsize=12)
    plt.ylabel('Test MAE', fontsize=12)
    plt.title('Swarm Allocation Comparison (Budget=500)', fontsize=14, fontweight='bold')
    plt.xticks(x, names)
    plt.grid(True, alpha=0.3, axis='y')
    plt.tight_layout()
    plt.savefig('swarm_allocation_comparison.png', dpi=300)
    plt.show()


def plot_coefficient_comparison(results):
    """
    Plot comparison of different PSO coefficient settings.

    Args:
        results: Dictionary with coefficient experiment results
    """
    names = list(results.keys())
    test_means = [results[name]['test_mae_mean'] for name in names]
    test_stds = [results[name]['test_mae_std'] for name in names]

    plt.figure(figsize=(12, 6))
    x = np.arange(len(names))
    plt.bar(x, test_means, yerr=test_stds, capsize=5, alpha=0.7, color='mediumseagreen')
    plt.xlabel('Coefficient Setting', fontsize=12)
    plt.ylabel('Test MAE', fontsize=12)
    plt.title('PSO Coefficient Comparison', fontsize=14, fontweight='bold')
    plt.xticks(x, names, rotation=45, ha='right')
    plt.grid(True, alpha=0.3, axis='y')
    plt.tight_layout()
    plt.savefig('coefficient_comparison.png', dpi=300)
    plt.show()


def create_results_table(results):
    """
    Create formatted results table for report.

    Args:
        results: Dictionary with experiment results
    """
    print("\n" + "="*80)
    print("RESULTS SUMMARY TABLE")
    print("="*80)

    for experiment_name, experiment_results in results.items():
        print(f"\n{experiment_name.upper().replace('_', ' ')}")
        print("-" * 80)
        print(f"{'Configuration':<40} {'Train MAE':<20} {'Test MAE':<20}")
        print("-" * 80)

        for config_name, config_results in experiment_results.items():
            train_str = f"{config_results['train_mae_mean']:.4f} ± {config_results['train_mae_std']:.4f}"
            test_str = f"{config_results['test_mae_mean']:.4f} ± {config_results['test_mae_std']:.4f}"
            print(f"{config_name:<40} {train_str:<20} {test_str:<20}")
        print()


def analyze_results(results_file='experiment_results.json'):
    """
    Load and analyze experimental results.

    Args:
        results_file: Path to JSON results file
    """
    with open(results_file, 'r') as f:
        results = json.load(f)

    # Print results table
    create_results_table(results)

    # Create visualizations
    plot_architecture_comparison(results['architecture'])
    plot_swarm_allocation(results['swarm_allocation'])
    plot_coefficient_comparison(results['coefficients'])

    # Find best configurations
    print("\n" + "="*80)
    print("BEST CONFIGURATIONS")
    print("="*80)

    for experiment_name, experiment_results in results.items():
        best_config = min(experiment_results.items(),
                         key=lambda x: x[1]['test_mae_mean'])
        print(f"\n{experiment_name.upper().replace('_', ' ')}:")
        print(f"  Best: {best_config[0]}")
        print(f"  Test MAE: {best_config[1]['test_mae_mean']:.4f} ± {best_config[1]['test_mae_std']:.4f}")


if __name__ == "__main__":
    # Analyze results if they exist
    try:
        analyze_results()
    except FileNotFoundError:
        print("No results file found. Run main.py first to generate results.")


Loading and preprocessing data...


FileNotFoundError: [Errno 2] No such file or directory: 'Concrete_Data.csv'

In [3]:
import numpy as np

class PSO:
    """
    Particle Swarm Optimization implementation following Algorithm 39
    from "Essentials of Metaheuristics" by Sean Luke.
    Uses informants topology rather than global best.
    """

    def __init__(self, fitness_function, dimensions, bounds,
                 swarm_size=30, max_iterations=100,
                 num_informants=3, w=0.729, c1=1.49445, c2=1.49445):
        """
        Initialize PSO with specified parameters.

        Args:
            fitness_function: Function to minimize (lower is better)
            dimensions: Number of dimensions in solution space
            bounds: Tuple of (min, max) for parameter values
            swarm_size: Number of particles in swarm
            max_iterations: Maximum number of iterations
            num_informants: Number of informants per particle
            w: Inertia weight
            c1: Cognitive coefficient (personal best influence)
            c2: Social coefficient (neighborhood best influence)
        """
        self.fitness_function = fitness_function
        self.dimensions = dimensions
        self.bounds = bounds
        self.swarm_size = swarm_size
        self.max_iterations = max_iterations
        self.num_informants = num_informants
        self.w = w  # Inertia weight
        self.c1 = c1  # Cognitive coefficient
        self.c2 = c2  # Social coefficient

        # For tracking best solution found
        self.global_best_position = None
        self.global_best_fitness = float('inf')
        self.fitness_history = []

    def initialize_swarm(self):
        """
        Initialize particle positions and velocities.
        Corresponds to lines 1-2 of Algorithm 39.

        Returns:
            positions: Random initial positions
            velocities: Random initial velocities
        """
        # Line 1: Initialize positions uniformly in search space
        positions = np.random.uniform(
            self.bounds[0],
            self.bounds[1],
            (self.swarm_size, self.dimensions)
        )

        # Line 2: Initialize velocities
        velocity_range = (self.bounds[1] - self.bounds[0]) * 0.1
        velocities = np.random.uniform(
            -velocity_range,
            velocity_range,
            (self.swarm_size, self.dimensions)
        )

        return positions, velocities

    def create_informants(self):
        """
        Create informant topology (ring topology).
        Each particle is influenced by k neighbors.
        Corresponds to line 3 of Algorithm 39.

        Returns:
            List of lists, where informants[i] contains indices of particle i's informants
        """
        # Line 3: Assign informants to particles
        informants = []
        for i in range(self.swarm_size):
            # Ring topology: include neighbors on both sides
            neighbors = []
            for j in range(1, self.num_informants + 1):
                left = (i - j) % self.swarm_size
                right = (i + j) % self.swarm_size
                neighbors.append(left)
                if len(neighbors) < self.num_informants:
                    neighbors.append(right)
            # Include self and limit to num_informants
            neighbors = list(set([i] + neighbors[:self.num_informants-1]))
            informants.append(neighbors)
        return informants

    def optimize(self, verbose=True):
        """
        Main PSO optimization loop.
        Follows Algorithm 39 from Essentials of Metaheuristics.

        Returns:
            best_position: Best solution found
            best_fitness: Fitness of best solution
        """
        # Lines 1-3: Initialize swarm
        positions, velocities = self.initialize_swarm()
        informants = self.create_informants()

        # Line 4: Initialize personal bests
        personal_best_positions = positions.copy()
        personal_best_fitness = np.array([self.fitness_function(p) for p in positions])

        # Initialize neighborhood bests
        neighborhood_best_positions = np.zeros_like(positions)
        neighborhood_best_fitness = np.full(self.swarm_size, float('inf'))

        # Track global best
        best_idx = np.argmin(personal_best_fitness)
        self.global_best_position = personal_best_positions[best_idx].copy()
        self.global_best_fitness = personal_best_fitness[best_idx]

        # Line 5: Main loop
        for iteration in range(self.max_iterations):

            # Update neighborhood bests for each particle
            for i in range(self.swarm_size):
                # Find best among informants
                informant_fitness = [personal_best_fitness[j] for j in informants[i]]
                best_informant_idx = informants[i][np.argmin(informant_fitness)]
                neighborhood_best_positions[i] = personal_best_positions[best_informant_idx]
                neighborhood_best_fitness[i] = personal_best_fitness[best_informant_idx]

            # Update each particle
            for i in range(self.swarm_size):

                # Line 6: Generate random values
                rp = np.random.uniform(0, 1, self.dimensions)
                rg = np.random.uniform(0, 1, self.dimensions)

                # Line 7: Update velocity
                # v_i = w * v_i + c1 * rp * (p_i - x_i) + c2 * rg * (n_i - x_i)
                cognitive = self.c1 * rp * (personal_best_positions[i] - positions[i])
                social = self.c2 * rg * (neighborhood_best_positions[i] - positions[i])
                velocities[i] = self.w * velocities[i] + cognitive + social

                # Line 8: Update position
                positions[i] = positions[i] + velocities[i]

                # Line 9: Enforce bounds (clamping strategy)
                positions[i] = np.clip(positions[i], self.bounds[0], self.bounds[1])

                # Line 10: Evaluate fitness
                fitness = self.fitness_function(positions[i])

                # Line 11: Update personal best
                if fitness < personal_best_fitness[i]:
                    personal_best_positions[i] = positions[i].copy()
                    personal_best_fitness[i] = fitness

                    # Update global best if necessary
                    if fitness < self.global_best_fitness:
                        self.global_best_position = positions[i].copy()
                        self.global_best_fitness = fitness

            # Store best fitness for this iteration
            self.fitness_history.append(self.global_best_fitness)

            if verbose and (iteration + 1) % 10 == 0:
                print(f"Iteration {iteration + 1}/{self.max_iterations}, "
                      f"Best Fitness: {self.global_best_fitness:.4f}")

        return self.global_best_position, self.global_best_fitness